
<img align="left" style="padding-right:100px;" src="http://pvexpect.com/Driftsledelse/Dagligvarer/nmbu.png">

  **Digital Driftsledelse - IND310**

 Jesper Ørntoft Frausig, 2022


<a name="3.0"></a>
# **3.0 Linjediagrammer**


- [3.0 Linjediagrammer](#3.0)
- [3.1 Fylt trinndiagram](#3.1)
- [3.2 Linjediagram med konfidensintervall](#3.2)
- [3.3 Linjediagram med lagdelte aggregater](#3.3)
- [3.4 Linjediagram med prosentakse](#3.4)
- [3.5 Linjediagram med punkter](#3.5)
- [3.6 Linjediagram med sekvensgenerator](#3.6)
- [3.7 Linjediagram med varierende størrelse](#3.7)
- [3.8 Linjediagram med flere serier](#3.8)
- [3.9 Skråningsgraf](#3.9)
- [3.10 Trinndiagram](#3.10)


<a name="3.1"></a>
## **3.1 Fylt trinndiagram**

https://altair-viz.github.io/gallery/filled_step_chart.html

In [1]:
!pip install yfinance

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 62 kB 1.7 MB/s 
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0


In [2]:
#Hent aktier
import yfinance as yf
def stockDownload(symbol):
  stock = yf.Ticker(symbol)
  hist = stock.history(period="1y")
  return hist

In [3]:
data=stockDownload("Mowi.OL")
data['Dato']=data.index

In [4]:
import altair as alt


alt.Chart(data).mark_area(
    color="lightblue",
    interpolate='step-after',
    line=True
).encode(
    x='Dato:T',
    y='Close:Q'
)

alt.Chart(...)

<a name="3.2"></a>
## **3.2 Linjediagram med konfidensintervall**

https://altair-viz.github.io/gallery/line_with_ci.html

In [6]:
import altair as alt
import datetime
import pandas as pd

data=pd.read_csv("https://www.fiskeridir.no/Akvakultur/Tall-og-analyse/Biomassestatistikk/Biomassestatistikk-etter-fylke/_/attachment/download/422fa084-c905-43b4-9446-a050e92e80e5:1c81cbe34307c35cb79e7dc5a55aae783ab17ac7/biostat-total-flk-2019.csv", delimiter=";", error_bad_lines=False, encoding='latin-1', decimal=",")
data_selection=data[(data.ARTSID=="LAKS")&(data['FYLKE']=='01 Finnmark')].groupby(['MÅNED','ÅR','MÅNED_KODE']).sum()
data_selection.reset_index(inplace=True)


data_selection=data_selection.assign(**{'Date':0, 'DAY':1})

for index, måned in enumerate(data_selection.MÅNED):
  
  data_selection.MÅNED[index]=str(måned).replace('DESEMBER','DECEMBER').replace('FEBRUAR','FEBRUARY').replace('JANUAR','JANUARY').replace('JULI','JULY').replace('JUNI','JUNE').replace('MAI','MAY').replace('MARS','MARCH').replace('OKTOBER','OCTOBER')

  data_selection.Date[index] =datetime.datetime.strptime(str(data_selection.ÅR[index])+str(data_selection.MÅNED[index])+str(data_selection.DAY[index]), "%Y%B%d")

data_selection.MÅNED=data_selection.MÅNED.astype(str)
data_selection['timestamp'] = pd.to_datetime(data_selection.Date)

base = alt.Chart(data_selection.drop(['Date','MÅNED'], axis=1)).encode(x="ÅR:O").properties(width=500)

bar = base.mark_line().encode(
    y="mean(BIOMASSE_KG):Q",
    #color="FYLKE:N"
)

band = base.mark_errorband(extent='ci').encode(
    x='ÅR:O',
    y=alt.Y('BIOMASSE_KG:Q', title='BIOMASSE_KG'),
)

alt.layer(bar, band)


/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:3326: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  exec(code_obj, self.user_global_ns, self.user_ns)
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()


alt.LayerChart(...)

<a name="3.3"></a>
## **3.3 Linjediagram med lagdelte aggregater**

https://altair-viz.github.io/gallery/layer_line_color_rule.html

In [7]:
#Installere bibliotekker
!pip install yfinance
!pip install pandas-datareader
!pip install yahoofinancials
!pip install dbnomics
!pip install dask[dataframe]
!pip install d6tstack

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for yahoofinancials: filename=yahoofinancials-1.6-py3-none-any.whl size=15192 sha256=d8860e65ddf8ce88359577965635e0350c779ab9c86b85c7713b48745d90b420
  Stored in directory: /root/.cache/pip/wheels/4b/63/46/e7110bfee88685fe69e338d1b14d1748921862aa57b6705b60
Successfully built yahoofinancials
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [8]:
#Hent aktier
import pandas as pd
import yfinance as yf
def stockDownload(symbol):
  stock = yf.Ticker(symbol)
  hist = stock.history(period="5y")
  return hist

In [9]:
#Hent aktie symboler
import pandas as pd
url="https://raw.githubusercontent.com/jfrausig/test/master/dagligvarer_ticker.csv" # dk_ticker #trend_bb_4
df=pd.read_csv(url, error_bad_lines=False)
aktier=[]
for index, row in df.iterrows():
    d=row.to_dict()
    aktier.append(d)
aktier=pd.DataFrame(aktier)
aktier['ticker'] = [x.upper() for x in aktier.ticker]




In [10]:

!mkdir stock_dfs

In [11]:
import csv
import os
import yfinance as yf
import pandas as pd
import time
start = time.time()

ticker_list = []

ticker_list=list(aktier.ticker)

data = yf.download(
        tickers = ticker_list,
        period = '2y',
        interval = '1wk',
        group_by = 'ticker',
        auto_adjust = False,
        prepost = False,
        threads = True,
        proxy = None
    )

data = data.T

#for ticker in aktier.ticker:
for i, ticker in enumerate(aktier.ticker): 
    aktie=pd.DataFrame(aktier[aktier.ticker==ticker].Virksomhed)['Virksomhed'][i]
    try:
      data.loc[(ticker,),].T.to_csv('stock_dfs/' + aktie + '.csv', sep=',', encoding='utf-8')
    except:
      print(str(aktie)+" try failed")
print('It took', time.time()-start, 'seconds.')

[*********************100%***********************]  12 of 12 completed

4 Failed downloads:
- SALMAR: No data found, symbol may be delisted
- SALMON.OL: No data found, symbol may be delisted
- SSC.OL: No data found, symbol may be delisted
- ORKLA: No data found, symbol may be delisted
It took 1.0361328125 seconds.


/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:925: PerformanceWarning: indexing past lexsort depth may impact performance.
  return self._getitem_tuple(key)


In [12]:
#sammenfør virksomheds info
import glob
import d6tstack.combine_csv
import dask.dataframe as dd
cfg_fnames = list(glob.glob('stock_dfs/*.csv'))
c = d6tstack.combine_csv.CombinerCSV(cfg_fnames)

# check columns
print('all equal',c.is_all_equal())
print('')
print(c.is_column_present())

fnames = d6tstack.combine_csv.CombinerCSV(cfg_fnames).to_csv_align(output_dir='test_data/output/')


# consistent format
virksomheds_info = dd.read_csv('test_data/output/*.csv',dtype={'ask': 'float64',
})
aktie_tidsserie=virksomheds_info.compute()

sniffing columns ok
all equal True

                                           Date  Open  High   Low  Close  Adj Close  Volume
file_path                                                                                  
stock_dfs/Andfjord_Salmon.csv              True  True  True  True   True       True    True
stock_dfs/Austevoll Seafood.csv            True  True  True  True   True       True    True
stock_dfs/Bakkafrost.csv                   True  True  True  True   True       True    True
stock_dfs/Grieg Seafood.csv                True  True  True  True   True       True    True
stock_dfs/Lerøy Seafood Group.csv          True  True  True  True   True       True    True
stock_dfs/Mowi.csv                         True  True  True  True   True       True    True
stock_dfs/Norway Royal Salmon.csv          True  True  True  True   True       True    True
stock_dfs/Orkla Group.csv                  True  True  True  True   True       True    True
stock_dfs/SalMar.csv                       T

In [13]:
import altair as alt
from vega_datasets import data



base = alt.Chart(aktie_tidsserie).properties(width=550)

line = base.mark_line().encode(
    x='Date:O',
    y='Close',
    color='filename',
    tooltip=['High:Q','Close:Q','Open:Q','Low:Q','Adj Close:Q','Volume:Q','filename:N']
)

rule = base.mark_rule().encode(
    y='average(Close)',
    color='filename',
    size=alt.value(2)
)

(line + rule) #.save('Linjediagrammer.html')

alt.LayerChart(...)

<a name="3.4"></a>
## **3.4 Linjediagram med prosentakse**

https://altair-viz.github.io/gallery/line_percent.html

In [14]:
data=pd.read_csv("https://www.fiskeridir.no/Akvakultur/Tall-og-analyse/Biomassestatistikk/Biomassestatistikk-etter-fylke/_/attachment/download/422fa084-c905-43b4-9446-a050e92e80e5:1c81cbe34307c35cb79e7dc5a55aae783ab17ac7/biostat-total-flk-2019.csv", delimiter=";", error_bad_lines=False, encoding='latin-1', decimal=",")
data_selection=data[(data.ARTSID=="LAKS")].groupby(['ÅR']).sum()
data_selection.reset_index(inplace=True)


alt.Chart(data_selection).transform_fold(['UTTAK_STK','DØDFISK_STK','UTKAST_STK','RØMMING_STK','ANDRE_STK']).transform_joinaggregate(
    TotalAmount='sum(BEHFISK_STK):Q',
).transform_calculate(
    PercentOfTotal="datum.value / datum.TotalAmount"
).mark_line().encode(
    alt.Y('PercentOfTotal:Q', axis=alt.Axis(format='.2%')),
    x='ÅR:N',
    color='key:N'
)



/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:3326: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  exec(code_obj, self.user_global_ns, self.user_ns)


alt.Chart(...)

<a name="3.5"></a>
## **3.5 Linjediagram med punkter**

https://altair-viz.github.io/gallery/line_chart_with_points.html

In [15]:
#Hent aktier
import yfinance as yf
def stockDownload(symbol):
  stock = yf.Ticker(symbol)
  hist = stock.history(period="1y")
  return hist

data=stockDownload("Mowi.OL")
data['Dato']=data.index

In [16]:
data

,Open,High,Low,Close,Volume,Dividends,Stock Splits,Dato
Date,,,,,,,,
2021-10-28 00:00:00+02:00,243.236742,245.398844,240.288418,241.369476,629411,0.0,0,2021-10-28 00:00:00+02:00
2021-10-29 00:00:00+02:00,240.583271,243.531595,239.403944,240.386719,668193,0.0,0,2021-10-29 00:00:00+02:00
2021-11-01 00:00:00+01:00,240.484984,242.253981,237.143556,240.583267,627901,0.0,0,2021-11-01 00:00:00+01:00
2021-11-02 00:00:00+01:00,237.831489,239.600487,233.900390,236.553879,856223,0.0,0,2021-11-02 00:00:00+01:00
2021-11-03 00:00:00+01:00,236.062486,239.502198,236.062486,238.617706,671633,0.0,0,2021-11-03 00:00:00+01:00
...,...,...,...,...,...,...,...,...
2022-10-24 00:00:00+02:00,140.000000,140.250000,134.850006,137.149994,984900,0.0,0,2022-10-24 00:00:00+02:00
2022-10-25 00:00:00+02:00,137.800003,140.899994,136.899994,140.899994,899816,0.0,0,2022-10-25 00:00:00+02:00
2022-10-26 00:00:00+02:00,146.000000,149.000000,144.649994,147.250000,1945602,0.0,0,2022-10-26 00:00:00+02:00


In [17]:
alt.Chart(data).mark_line(point=True).encode(
    x='Dato',
    y='High'
)

alt.Chart(...)

<a name="3.6"></a>
## **3.6 Linjediagram med sekvensgenerator**

https://altair-viz.github.io/gallery/line_chart_with_generator.html

<a name="3.7"></a>
## **3.7 Linjediagram med varierende størrelse**

https://altair-viz.github.io/gallery/trail_marker.html

In [18]:
data=pd.read_csv("https://www.fiskeridir.no/Akvakultur/Tall-og-analyse/Biomassestatistikk/Biomassestatistikk-etter-fylke/_/attachment/download/422fa084-c905-43b4-9446-a050e92e80e5:1c81cbe34307c35cb79e7dc5a55aae783ab17ac7/biostat-total-flk-2019.csv", delimiter=";", error_bad_lines=False, encoding='latin-1', decimal=",")
data_selection=data[(data.ARTSID=="LAKS")].groupby(['ÅR']).sum()
data_selection.reset_index(inplace=True)


alt.Chart(data_selection).transform_fold(['UTTAK_STK','DØDFISK_STK','UTKAST_STK','RØMMING_STK','ANDRE_STK']).transform_joinaggregate(
    TotalAmount='sum(BEHFISK_STK):Q',
).transform_calculate(
    PercentOfTotal="datum.value / datum.TotalAmount"
).mark_trail().encode(
    alt.Y('PercentOfTotal:Q', axis=alt.Axis(format='.2%')),
    x='ÅR:N',
    color='key:N',
    size='PercentOfTotal:Q'
)



/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:3326: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  exec(code_obj, self.user_global_ns, self.user_ns)


alt.Chart(...)

<a name="3.8"></a>
## **3.8 Linjediagram med flere serier**

https://altair-viz.github.io/gallery/multi_series_line.html

In [19]:
import altair as alt
from vega_datasets import data



alt.Chart(aktie_tidsserie).mark_line().encode(
    x='Date:O',
    y='Close',
    color='filename',
    strokeDash='filename'
).properties(width=550)



alt.Chart(...)

<a name="3.9"></a>
## **3.9 Skråningsgraf**

https://altair-viz.github.io/gallery/slope_graph.html

In [20]:
import altair as alt
from vega_datasets import data

source = data.barley()

alt.Chart(source).mark_line().encode(
    x='year:O',
    y='median(yield)',
    color='site'
)

alt.Chart(...)

<a name="3.10"></a>
## **3.10 Trinndiagram**

https://altair-viz.github.io/gallery/step_chart.html

In [23]:
#Hent aktier
import yfinance as yf
def stockDownload(symbol):
  stock = yf.Ticker(symbol)
  hist = stock.history(period="12wk")
  return hist

data=stockDownload("Mowi.OL")
data['Dato']=data.index

alt.Chart(data).mark_line(interpolate='step-after').encode(
    x='Dato',
    y='Low'
)

alt.Chart(...)

In [ ]:
"""
import altair as alt
data = pd.read_csv('http://pvexpect.com/Data_forst%C3%A5else_akvakultur_data/akvakultur_data.csv')
data.head(5)

alt.Chart(data).mark_circle(size=60).encode(
    x='Req to dissolved oxygen:Q',
    y='Req to Living space:Q',
    color='Fisk',
    size='Pris',
    tooltip=['Fisk og fiske art']
).interactive()
"""

alt.Chart(...)